Authored by Felix Last <<mail@felixlast.de>> to summarize the method presented in [Training PPA Models for Embedded Memories on a Low-data Diet](https://doi.org/10.1145/3556539).

This code was written using Python 3.11 and [pipenv](https://pipenv.pypa.io/en/latest/). Install both to run.

In [1]:
from IPython.display import display # can be replaced with print outside of jupyter
import numpy as np
import pandas as pd
import sklearn.model_selection
import sklearn.preprocessing
import tensorflow as tf

2023-05-03 18:53:17.452300: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 18:53:17.458415: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 18:53:17.581277: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 18:53:17.584171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 18:53:19.923492: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# define stub data

# X source domain
x_raw = pd.DataFrame({
    "column_mux": [2,2,2,4,4,4],
    "bank": [1,1,1,2,2,2],
    "nw": [1024,2048,4096,1024,2048,4096],
    "nb": [100,200,100,200,100,200],
})
display(x_raw)
# X target domain
x_raw_target = pd.DataFrame({
    "column_mux": [2,2,2,4,4,4],
    "bank": [2,2,2,1,1,1],
    "nw": [1024,2048,4096,1024,2048,4096],
    "nb": [100,200,100,200,100,200],
})
display(x_raw_target)

# Y source domain
y_raw = pd.DataFrame({
    "area": [100.0, 200.0, 300.0, 400.0, 500.0, 600.0],
})
display(y_raw)

# Y target domain
y_raw_target = pd.DataFrame({
    "area": [100.0, 200.0, 300.0, 400.0, 500.0, 600.0],
})
display(y_raw_target)

,column_mux,bank,nw,nb
0,2,1,1024,100
1,2,1,2048,200
2,2,1,4096,100
3,4,2,1024,200
4,4,2,2048,100
5,4,2,4096,200


,column_mux,bank,nw,nb
0,2,2,1024,100
1,2,2,2048,200
2,2,2,4096,100
3,4,1,1024,200
4,4,1,2048,100
5,4,1,4096,200


,area
0,100.0
1,200.0
2,300.0
3,400.0
4,500.0
5,600.0


,area
0,100.0
1,200.0
2,300.0
3,400.0
4,500.0
5,600.0


In [3]:
# define preprocessors
x_scaler = sklearn.preprocessing.StandardScaler()
y_scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 1))

In [4]:
# fit preprocessors on joint data distribution
x_union = pd.concat([x_raw, x_raw_target], ignore_index=True, axis=0)
x_scaler.fit(x_union)
y_union = pd.concat([y_raw, y_raw_target], ignore_index=True, axis=0)
y_scaler.fit(y_union)
print(x_scaler.mean_, x_scaler.scale_)
print(y_scaler.data_min_, y_scaler.data_max_)

[3.00000000e+00 1.50000000e+00 2.38933333e+03 1.50000000e+02] [1.00000000e+00 5.00000000e-01 1.27715239e+03 5.00000000e+01]
[100.] [600.]


In [5]:
# preprocess data
x = x_scaler.transform(x_raw)
x_target = x_scaler.transform(x_raw_target)
y = y_scaler.transform(y_raw)
y_target = y_scaler.transform(y_raw_target)

In [6]:
# split test / train data
x_target_train, x_target_test, y_target_train, y_target_test = sklearn.model_selection.train_test_split(
    x_target, y_target, test_size=0.33, random_state=1
)

In [7]:
# define model (very simple FFNN, architecture needs problem-specific fine-tuning)
input_layer = tf.keras.Input(shape=(x_union.shape[1],))
h1 = tf.keras.layers.Dense(32, activation=tf.nn.relu)(input_layer)
output_layer = tf.keras.layers.Dense(y_union.shape[1], activation=tf.nn.relu)(h1)
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [8]:
# compile model, setting optimizer & loss function
model.compile(
    optimizer="adam",
    loss="log_cosh",
    metrics=None,
)

In [9]:
# pre-train
model.fit(
    x=x, 
    y=y,
    batch_size=None,
    epochs=1,
    verbose="auto",
    callbacks=None, # TODO: add early stopping, tensorboard
    validation_split=0.0,
    shuffle=True,
)

1/1 [==============================] - 1s 1s/step - loss: 0.1647


In [10]:
# fine-tune
model.fit(
    x=x_target_train, 
    y=y_target_train,
    batch_size=None,
    epochs=1,
    verbose="auto",
    callbacks=None, # TODO: add early stopping, tensorboard
    validation_split=0.0,
    shuffle=True,
)

1/1 [==============================] - 0s 14ms/step - loss: 0.1786


In [11]:
# evaluate accuracy (estimation error) on target domain
y_pred = model.predict(
    x_target_test
)
# reverse scale predictions
y_pred_orig = y_scaler.inverse_transform(y_pred)
y_true_orig = y_scaler.inverse_transform(y_target_test)
residuals = y_true_orig - y_pred_orig
display(y_pred_orig)
display(y_true_orig)
display(residuals)

1/1 [==============================] - 0s 355ms/step


array([[100.],
       [100.]], dtype=float32)

array([[300.],
       [200.]])

array([[200.],
       [100.]])

In [12]:
# relative metric suitable for evaluating when ground truth values strictly positive
# not suitable as loss function
# should be used on original data scale

def symmetric_rel_err(y_true, y_pred):
    """Unsigned version of signed symmetric percentage bias (Morley et al.,
    2018). Interpretable as a percentage, but symmetric unlike APE.
    """
    return (np.exp(np.abs(np.log(y_pred / y_true))) - 1) * 100

def median_symmetric_rel_err(y_true, y_pred, axis=0):
    """Computes median of `symmetric_rel_err` across observations"""
    return np.median(symmetric_rel_err(y_true, y_pred), axis=axis)

def mean_median_symmetric_rel_err(y_true, y_pred):
    """Computes variable-wise median `symmetric_rel_err` before aggregating
    across variables using mean."""
    return np.mean(median_symmetric_rel_err(y_true, y_pred))

mean_median_symmetric_rel_err(y_pred_orig, y_true_orig)

149.99999999999997